In [1]:
%reset -f

In [ ]:
##############################
# FD_simple_CNN_ncwork_float32
# CNN code for image_data which is stored as float32
##############################

In [1]:
###############################
# IMPORTS
###############################

# torch things
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import Sampler
from torch.utils.data.sampler import WeightedRandomSampler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchsummary import summary

# netCDF4 things
import netCDF4 as nc
from netCDF4 import Dataset as ncDataset
import xarray as xr

# normal python things
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import copy


In [2]:
###############################
# CNN ARCHITECTURE
###############################

# Create simple CNN architecture
class tinyCNN(nn.Module):
    def __init__(self, in_channels, out_features):
        super(tinyCNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels=4, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution, meaning that the output size will be the same dimensions as the input (294x294) here
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution, meaning that the output size will be the same dimensions as the input (294x294) here
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv6 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(128*4*4, out_features) # size should be (out_channels from previous layer)*(input_size/2^num_layers) because the max pooling reduces dimension by 2 fold in each layer
        # self.softplus = nn.Softplus()  # Define Softplus as a layer

        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = F.relu(self.conv5(x))
        x = self.pool(x)
        x = F.relu(self.conv6(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1) # flattens to be able to put into the fully-connected layer
        x = self.fc1(x)
        x = torch.abs(x)  # Ensure no negative values in the output (do EITHER this or softplus, not both)
        # x = self.softplus(x) # Ensure no negative values in output (do EITHER this or abs, not both)

        return x
    
###############################
# SET GPU AND HYPERPARAMETERS
###############################

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Hyperparameters
in_channels = 1
out_features = 1
learning_rate = 0.001
batch_size = 512
num_epochs = 100
input_size = (1,294,294) #[C, H, W], the np arrays of images are [H, W], and the transform function converts to [H, W, C] 

# network structure
# in_channels = [

print("hyperparameters set")

###############################
# INITIALIZE NETWORK
###############################

# Initialize network
model = tinyCNN(in_channels, out_features).to(device)

# Print the model summary
summary(model, input_size=input_size)

Using device: cuda
hyperparameters set
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 4, 294, 294]              40
         MaxPool2d-2          [-1, 4, 147, 147]               0
            Conv2d-3          [-1, 8, 147, 147]             296
         MaxPool2d-4            [-1, 8, 73, 73]               0
            Conv2d-5           [-1, 16, 73, 73]           1,168
         MaxPool2d-6           [-1, 16, 36, 36]               0
            Conv2d-7           [-1, 32, 36, 36]           4,640
         MaxPool2d-8           [-1, 32, 18, 18]               0
            Conv2d-9           [-1, 64, 18, 18]          18,496
        MaxPool2d-10             [-1, 64, 9, 9]               0
           Conv2d-11            [-1, 128, 9, 9]          73,856
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Linear-13                    [-1, 1]           2,049


In [8]:
###############################
# TRANSFORMATION DEFINITIONS
###############################

# define the desired transformations to the images and the labels (transform, transform_label)

# transform: transforms images from numpy array of shape [H, W] to tensors of shape [C, H, W] in range [0, 1]
transform = transforms.Compose([
    transforms.ToTensor()  # Converts images to PyTorch tensors and scales to [0, 1]
])


# # transform_label: transforms the labels to a more normal distribution
# def transform_label_function(label):
#     # Ensure label is a tensor
#     label_tensor = torch.tensor(label, dtype=torch.float32)
#     # Apply the transformation
#     transformed_label = (torch.log10(label_tensor + 0.001) + 2)
#     return transformed_label

# transform_label: transforms the labels to be between [0,1]
fracture_density_max = xr.open_dataset('data/data/orig_nc_files/ncfile_comb_north.nc')['fracture_density'].max()
fracture_density_min = xr.open_dataset('data/data/orig_nc_files/ncfile_comb_north.nc')['fracture_density'].min()
print(fracture_density_max)
print(fracture_density_min)
def transform_label_function(label):
    # Ensure label is a tensor
    label_tensor = torch.tensor(label, dtype=torch.float32)
    # Apply the transformation
    transformed_label = torch.tensor(((label_tensor-fracture_density_min) / (fracture_density_max-fracture_density_min)).values, dtype=torch.float32)
    return transformed_label

# inverse transform (to get back to original fracture density after the training)
# be careful to make sure this is the inverse function to the transform function (manual)
# def inverse_transform_label_function(transformed_label):
#     inverse_transformed_label = 10**(transformed_label-2) - 0.001
#     return inverse_transformed_label

# inverse transform for between [0,1] transform
def inverse_transform_label_function(transformed_label):
    inverse_transformed_label = torch.tensor(((transformed_label*((fracture_density_max-fracture_density_min))) - fracture_density_min).values, dtype=torch.float32)
    return inverse_transformed_label




<xarray.DataArray 'fracture_density' ()> Size: 8B
array(0.45724004)
<xarray.DataArray 'fracture_density' ()> Size: 8B
array(0.)


In [ ]:
# # #################################################################
# # # TO LOOK AT THE LABELS AND TRANSFORMED LABELS IN THE DATASET
# # #################################################################

# # extract original labels:
# nc_data = ncDataset('data/data/ncfile_south_combined.nc', 'r')
# original_labels = nc_data.variables['fracture_density'][:].astype(np.float32)

# # Compute transformed labels
# transformed_labels = []
# for label in original_labels:
#     transformed_label = transform_label_function(label)
#     transformed_labels.append(transformed_label)

# # Convert list to numpy arrays
# transformed_labels = np.array(transformed_labels)

# # histogram plots
# plt.figure(figsize=(12, 6))

# # Histogram of original labels
# plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
# plt.hist(original_labels, bins=200, alpha=0.7, color='blue')
# plt.title('Original Labels Distribution')
# plt.xlabel('Fracture Density')
# plt.ylabel('Frequency')
# plt.grid(True)

# # Histogram of transformed labels
# plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
# plt.hist(transformed_labels, bins=200, alpha=0.7, color='green')
# plt.title('Transformed Labels Distribution')
# plt.xlabel('Transformed Fracture Density')
# plt.grid(True)

# plt.tight_layout()
# plt.show()

# # how many FD=0 are in the dataset:
# num_zeros = np.count_nonzero(original_labels == 0)
# print("Number of zeros in the original label set:", num_zeros)

In [8]:
###############################
# DATASET CLASS CREATION FROM .nc FILES
###############################

# create custom dataset class from the .nc files
class NCDataset(Dataset):
    def __init__(self, nc_file_path, transform=None, transform_label=None):
        """
        Args:
            nc_file_path (string): Path to the .nc file
            transform (callable, optional): optional transform to apply to images
            transform_label (callable, optional): optional transform to apply to the labels
        """
        # load the .nc file
        self.nc_data = ncDataset(nc_file_path, 'r')
        
        # access the images (image_data) and labels (fracture_density) from the .nc file
        self.image_data = self.nc_data.variables['image_data']
        self.fracture_density = self.nc_data.variables['fracture_density']
        
        # image and label transforms (transform, transform_label)
        self.transform = transform
        self.transform_label = transform_label
        
        # load whole array into memory (could also convert to float32 here if that is not already done in preprocessing)
        self.image_data = self.image_data[:,:,:].astype(np.float32)
        self.fracture_density = self.fracture_density[:].astype(np.float32)
        
    def __len__(self):
        return self.image_data.shape[2]  # number of images is the third dimension [2]
        
    def __getitem__(self, idx):
        
        # Extract a single image and its corresponding label
        image = self.image_data[:, :, idx]
        label = self.fracture_density[idx]
        
        # Apply image transformation if any
        if self.transform:
            image = self.transform(image)
        
        # apply label transformation, if any
        if self.transform_label:
            label = self.transform_label(label)    
            
        # The image should already be a tensor because of the transform, 
        # and label needs to be converted only if transform_label does not do it
        if not torch.is_tensor(label):
            label = torch.tensor(label, dtype=torch.float32)
        
        # return
        return image, label
        


In [9]:
###############################
# SET THE DATALOADERS (this takes a while when using large dataset)
###############################

# .nc file path
nc_file_path = 'data/data/ncfile_south_outside_train.nc'
nc_file_path_val = 'data/data/ncfile_south_outside_val.nc'

# load dataset from .nc file using NCDataset
# ds_train = NCDataset(nc_file_path, transform=transform, transform_label=lambda x: transform_label_function(x))
# ds_val = NCDataset(nc_file_path_val, transform=transform, transform_label=lambda x: transform_label_function(x))
ds_train = NCDataset(nc_file_path, transform=transform, transform_label=transform_label_function)
ds_val = NCDataset(nc_file_path_val, transform=transform, transform_label=transform_label_function)

# set the sampler based on samples <= 0.05 (or some other value)
weights = torch.ones(len(ds_train), dtype=torch.float)  # Start with all weights equal to 1
zero_label_weight = 0.5  # Weight for zero-labeled samples, adjust as needed
labels_train = np.array(nc.Dataset(nc_file_path, 'r').variables['fracture_density']).astype(np.float32) 
weights[labels_train == 0] = zero_label_weight  # Assign lower weight to zero-labeled samples
sampler_zerosreduction = WeightedRandomSampler(weights=weights, num_samples=len(ds_train), replacement=False)

# dataloader (NOTE: only use either a sampler or shuffle, not both)
dataloader_train = DataLoader(ds_train, batch_size=batch_size, sampler=None, shuffle=True, num_workers=8)
dataloader_val = DataLoader(ds_val, batch_size=batch_size, sampler=None, num_workers=8)

print(f'DataLoaders for train and val sets have been set')

DataLoaders for train and val sets have been set


In [10]:
###############################
# LOSS, OPTIMIZER
###############################

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(f'loss criterion and optimizer have been set')

loss criterion and optimizer have been set


In [ ]:
image, label = ds_train[0]  # Get the first item
print(f"Label type after fetch: {type(label)}")  # Check the type directly

In [11]:
###############################
# TRAINING LOOP
###############################

losses = []  # List to store loss values
losses_train = []
losses_val = []

best_loss_val = float('inf')  # Initialize with a high value
best_loss_train = float('inf')  # Initialize with a high value

# beginning of training print statement:
print(f'train size[{len(ds_train)}]; val size[{len(ds_val)}]; batch size[{batch_size}]; batches per epoch[{len(dataloader_train)}]')

# Now use the DataLoader in the training loop
start_time_trainloop = time.time()
for epoch in range(num_epochs):
    # print(f"\r Starting epoch {epoch+1}")
    
    # start epoch timer
    start_time_epoch = time.time() 
    
    for batch_idx, (images, labels) in enumerate(dataloader_train):
        # print(f"\r Processing batch {batch_idx+1}")
        
        # put data to cuda if possible
        images = images.to(device=device) # this moves the data tensor to the device which will carry out the computation
        labels = labels.to(device=device) # this moves the target tensor to the device which will carry out the computation
        
        # forward pass
        preds = torch.squeeze(model(images)) # this does the forward pass of the data through the model and computes the model predictions (aka 'scores') for each imput in the batch. scores represents the model prediction for the given input data
        # preds = torch.squeeze(preds)  # squeezes dimensions from [64, 1] to [64]. Now preds has shape [64], which is the same shape as the labels 
        loss = criterion(preds, labels) # this calculates the loss between the model prediction (scores) and the true label value (targets)
        # print(f'preds shape[{preds.shape}]')
        
        # backward pass
        optimizer.zero_grad() # sets all gradients to zero at the beginning of each batch, so doesn't store the backprop calculations from previous forward props
        loss.backward() # computes the gradients via backprop

        # gradient descent or adam step
        optimizer.step() # updates the weights depending on the gradients computed in loss.backward

        # Record the training loss for each iteration
        losses.append(loss.item())
        
    # record the training loss for each epoch
    losses_train.append(loss.item())
        
    # Validation phase (get validation loss and collect labels and predictions)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
        for images, labels in dataloader_val:
            images = images.to(device=device)
            labels = labels.to(device=device)
            preds = torch.squeeze(model(images))
            loss_val = criterion(preds, labels)
    losses_val.append(loss_val.item())
    
    # save loss at the global minimum of the validation loss (loss_val)
    if loss_val < best_loss_val:
        # Update the best loss value
        best_loss_val = loss_val
        # Save the best model parameters
        bestmodel_val_wts = copy.deepcopy(model.state_dict())
        # Optionally, save to a file
        torch.save(bestmodel_val_wts, 'bestmodel_wts_val.pt')
        
    # save loss at the global minimum of the training loss (loss_train)
    if loss < best_loss_train:
        # Update the best loss value
        best_loss_train = loss
        # Save the best model parameters
        bestmodel_train_wts = copy.deepcopy(model.state_dict())
        # Optionally, save to a file
        torch.save(bestmodel_train_wts, 'bestmodel_wts_train.pt')
        
    # end epoch timer
    end_time_epoch = time.time()
    epoch_duration = end_time_epoch - start_time_epoch
    elapsed_time_training = (end_time_epoch-start_time_trainloop)/60 #[minutes]
    
    # print end-of-epoch statement
    sys.stdout.write(f'\r epoch[{epoch+1}/{num_epochs}]; train loss[{loss.item():.8f}]; val loss[{loss_val.item():.8f}]; time per epoch[{(elapsed_time_training/(epoch+1)):.4f} mins]; elapsed time[{elapsed_time_training:.4f} mins]')

    # Save the loss lists
    # Convert lists to tensors
    train_loss_tensor = torch.tensor(losses_train)
    val_loss_tensor = torch.tensor(losses_val)
    # Save tensors
    torch.save({'train_loss': train_loss_tensor, 'val_loss': val_loss_tensor}, 'losses.pt')
    
# NOTE: 'targets' means the ground-truth labels, 'scores' or 'preds' are model predictions

train size[5515]; val size[5515]; batch size[128]; batches per epoch[44]
 epoch[12/100]; train loss[0.00030389]; val loss[0.00437625]; elapsed time[0.8740 mins]

KeyboardInterrupt: 

In [ ]:
########################################
# AFTER THE TRAINING LOOP CONCLUDES:
# LOAD THE TRAINED MODEL
########################################

# After training is complete, you can load the best model weights back into your model
model = CNN(in_channels, out_features)
weights_path = 'bestmodel_wts_train.pt'
model.load_state_dict(torch.load(weights_path)) # load the model state from the desired output file (either best validation model or best training model)
model.to(device) # send model to GPU
model.eval()  # Evaluate model mode

In [ ]:
########################################
# AFTER THE TRAINING LOOP CONCLUDES:
# GET LABELS AND PREDICTIONS FOR TRAIN AND VAL SETS FROM THE TRAINED MODEL
########################################

# get labels and predictions for both training set and validation set
# training set:
model.eval()  # Evaluate mode
predictions_train = []
actual_labels_train = []
with torch.no_grad():
    for images, labels in dataloader_train:  # Assuming you want to do this for the training set
        images = images.to(device)
        preds = model(images)
        predictions_train.extend(preds.view(-1).tolist())  # Flatten and convert to list
        actual_labels_train.extend(labels.tolist())
# Ensure predictions and actual_labels are numpy arrays for easier manipulation
predictions_train = np.array(predictions_train)
actual_labels_train = np.array(actual_labels_train)

print(f"train set: predictions and labels acquired ({len(actual_labels_train)})")
    
# validation set:
model.eval()  # Evaluate mode
predictions_val = []
actual_labels_val = []
with torch.no_grad():
    for images_val, labels_val in dataloader_val:  # Assuming you want to do this for the training set
        images_val = images_val.to(device)
        preds_val = model(images_val)
        predictions_val.extend(preds_val.view(-1).tolist())  # Flatten and convert to list
        actual_labels_val.extend(labels_val.tolist())
# Ensure predictions and actual_labels are numpy arrays for easier manipulation
predictions_val = np.array(predictions_val)
actual_labels_val = np.array(actual_labels_val)

print(f"validation set: predictions and labels acquired ({len(actual_labels_val)})")

# get the training and validation RMSE (transformed and un-transformed)
rmse_train_transf = np.sqrt(np.mean((actual_labels_train - predictions_train) ** 2))
rmse_val_transf = np.sqrt(np.mean((actual_labels_val - predictions_val) ** 2))
# rmse_train_untransf = np.sqrt(np.mean((inverse_transform_label_function(actual_labels_train, facs=3) - inverse_transform_label_function(predictions_train, facs=3)) ** 2))
# rmse_val_untransf = np.sqrt(np.mean((inverse_transform_label_function(actual_labels_val, facs=3) - inverse_transform_label_function(predictions_val, facs=3)) ** 2))

# print(f'training set RMSE:\n rmse_train_transf[{rmse_train_transf:.4f}]\n rmse_val_transf[{rmse_val_transf:.4f}]\nvalidation set RMSE: \n rmse_train_untransf[{rmse_train_untransf:.4f}]\n rmse_val_untransf[{rmse_val_untransf:.4f}]')


In [ ]:
###############################
# PLOTS
###############################

# plot of training and validation losses across epochs
plt.figure()
plt.plot(np.asarray(loaded_losses['train_loss']), c='black', label='training loss')
plt.plot(np.asarray(loaded_losses['val_loss']), c='red', label='validation loss')
plt.legend()
plt.title(f'training and validation losses per epoch; {num_epochs} epochs')
plt.xlabel('epoch')
plt.ylabel('loss')
# plt.ylim(0,0.005)

# plot of the predictions versus labels for training set and validation set (transformed labels)
plt.figure()
plt.scatter(actual_labels_train, predictions_train, c='black', s=12, label='train set')
# plt.scatter(actual_labels_val, predictions_val, c='red', s=7, label='val set')
plt.plot(np.array(np.linspace(-2,2,11)),np.array(np.linspace(-2,2,11)),'b-')
plt.title('predictions vs actual labels')
plt.xlabel('actual (ground truth) labels')
plt.ylabel('predictions (CNN preds)')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.legend()

# plot of the predictions versus labels for training set and validation set (original labels)
# plt.figure()
# plt.scatter(inverse_transform_label_function(actual_labels_train), inverse_transform_label_function(predictions_train), c='black', s=12, label='train set')
# # plt.scatter(inverse_transform_label_function(actual_labels_val), inverse_transform_label_function(predictions_val), c='red', s=7, label='val set')
# plt.plot(np.array(np.linspace(-2,2,11)),np.array(np.linspace(-2,2,11)),'b-')
# plt.title('predictions vs actual labels')
# plt.xlabel('actual (ground truth) labels')
# plt.ylabel('predictions (CNN preds)')
# plt.xlim([0, 0.5])
# plt.ylim([0, 0.5])
# plt.legend()



In [ ]:
###############################
# PLOT TRAINING LOSS CURVE AT EACH ITERATION
###############################

# Calculate the total number of batches processed
total_batches = len(dataloader_train) * num_epochs

# plt.figure(figsize=(10, 5))
plt
plt.plot(losses, label='training Loss')

# # Add vertical lines for each epoch
# for epoch in range(num_epochs):
#     # Calculate the iteration index at the end of each epoch
#     epoch_end_iter = len(data_loader) * (epoch + 1)
#     plt.axvline(x=epoch_end_iter, color='r', linestyle='--', linewidth=1)

# plt.ylim(0,0.05)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('training loss as a function of iterations')
plt.legend()
plt.show()